In [14]:
import cv2
import numpy as np
import uuid

VIDEO_PATH = "C:\\Users\\sadiq\\Lapchole1.mp4"  # TODO: set to one of the challenge videos

# Lucas–Kanade optical flow params
lk_params = dict(
    winSize=(21, 21),
    maxLevel=3,
    criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 30, 0.01),
)

# Shi–Tomasi good features params
feature_params = dict(
    maxCorners=40,
    qualityLevel=0.01,
    minDistance=3,
    blockSize=7,
)

annotations = {}  # id -> dict with points, color, health
current_frame = None
prev_gray = None
selected_points = []  # temporary storage during mouse click

def on_mouse(event, x, y, flags, param):
    global annotations, current_frame
    if event == cv2.EVENT_LBUTTONDOWN and current_frame is not None:
        # Create a new annotation anchored at (x, y)
        ann_id = str(uuid.uuid4())
        h, w = current_frame.shape[:2]

        # Define a local ROI around click
        roi_size = 64
        x0 = max(x - roi_size // 2, 0)
        y0 = max(y - roi_size // 2, 0)
        x1 = min(x0 + roi_size, w)
        y1 = min(y0 + roi_size, h)

        roi = current_frame[y0:y1, x0:x1]
        gray_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)

        # Detect features in ROI
        pts = cv2.goodFeaturesToTrack(gray_roi, mask=None, **feature_params)
        if pts is not None:
            # Shift ROI coordinates to full-frame coordinates
            pts[:, 0, 0] += x0
            pts[:, 0, 1] += y0

            annotations[ann_id] = {
                "id": ann_id,
                "anchor": np.array([[x, y]], dtype=np.float32),
                "points_prev": pts.astype(np.float32),
                "points_init": pts.astype(np.float32).copy(),
                "color": tuple(np.random.randint(0, 255, size=3).tolist()),
                "health": 1.0,
            }
            print(f"Created annotation {ann_id} at ({x}, {y}) with {len(pts)} features")

def main():
    global current_frame, prev_gray, annotations

    cap = cv2.VideoCapture(VIDEO_PATH)
    if not cap.isOpened():
        print("Error: could not open video")
        return

    cv2.namedWindow("HoloRay Tracker Prototype")
    cv2.setMouseCallback("HoloRay Tracker Prototype", on_mouse)

    ret, frame = cap.read()
    if not ret:
        print("Error: empty video")
        return

    prev_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        current_frame = frame.copy()
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # For each annotation, track and update
        for ann_id, ann in list(annotations.items()):
            pts_prev = ann["points_prev"]
            if pts_prev is None or len(pts_prev) == 0:
                continue

            # Optical flow from prev_gray -> frame_gray
            pts_next, st, err = cv2.calcOpticalFlowPyrLK(
                prev_gray, frame_gray, pts_prev, None, **lk_params
            )

            if pts_next is None:
                ann["health"] = 0.0
                continue

            st = st.reshape(-1)
            good_old = pts_prev[st == 1]
            good_new = pts_next[st == 1]

            if len(good_old) < 4:
                # Not enough points to estimate transform robustly
                ann["health"] = len(good_old) / max(len(pts_prev), 1)
                ann["points_prev"] = good_new.reshape(-1, 1, 2)
            else:
                # Estimate affine transform with RANSAC
                M, inliers = cv2.estimateAffinePartial2D(
                    good_old, good_new, method=cv2.RANSAC, ransacReprojThreshold=3
                )

                if M is not None:
                    inliers = inliers.reshape(-1).astype(bool)
                    good_old_in = good_old[inliers]
                    good_new_in = good_new[inliers]

                    # Update anchor point with the transform
                    anchor = ann["anchor"].reshape(1, 1, 2)
                    anchor_new = cv2.transform(anchor, M.reshape(2, 3))
                    ann["anchor"] = anchor_new.reshape(1, 2)

                    # Update stored points
                    ann["points_prev"] = good_new_in.reshape(-1, 1, 2)

                    # Health: fraction of inliers
                    ann["health"] = float(len(good_new_in)) / float(len(pts_prev))
                else:
                    ann["health"] = 0.0

            # If health too low, consider this annotation lost (for now just dim it)
            health = ann["health"]
            ax, ay = int(ann["anchor"][0, 0]), int(ann["anchor"][0, 1])

            color = ann["color"]
            # Draw annotation: circle at anchor plus its ID and health
            alpha = max(0.2, min(1.0, health))
            draw_color = (int(color[0] * alpha), int(color[1] * alpha), int(color[2] * alpha))
            cv2.circle(current_frame, (ax, ay), 6, draw_color, -1)
            cv2.putText(
                current_frame,
                f"{ann_id[:4]} h={health:.2f}",
                (ax + 8, ay - 8),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.4,
                draw_color,
                1,
                cv2.LINE_AA,
            )

        cv2.imshow("HoloRay Tracker Prototype", current_frame)
        key = cv2.waitKey(1) & 0xFF
        if key == 27 or key == ord("q"):
            break

        prev_gray = frame_gray.copy()

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Created annotation 1191055f-c457-4d69-8d65-d6eb95d9f991 at (382, 273) with 26 features
Created annotation 692a0592-e5af-4130-ad7e-e8a54c3b5493 at (223, 215) with 15 features
Created annotation 913e86af-2b85-417f-b18d-c3d38c825165 at (659, 252) with 40 features
Created annotation 8740b49f-1280-4cd8-9c61-7768a8089ae0 at (739, 147) with 40 features
